# Step 1: Pull the data from the API

In [2]:
# import the libraries
import pandas as pd
import requests
import json
from config import headers
import numpy as np
import matplotlib as plt
import seaborn as sns

# Training ML model 

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from collections import Counter

In [4]:
# Import the csv file
merged_df = pd.read_csv('../Resources/nlp_model_data.csv')
merged_df.head()

,Unnamed: 0,level_0,index,review_title,review_comment,review_star_rating,asin,comments_no_stopwords,neg,neu,pos,compound,sentiment
0,0,0,0,Charges my bike computer very well.,This pack of usb-c to usb-c cables is perfectl...,5,B0D49CWPH2,This pack usb-c usb-c cables perfectly service...,0.034,0.633,0.333,0.9645,1
1,1,1,1,"Dependable, high-performance, longevity",The USB C to USB C Cable 5-Pack (6FT) is a rel...,5,B0D49CWPH2,The USB C USB C Cable 5-Pack (6FT) reliable ve...,0.000,0.781,0.219,0.9337,1
2,2,2,2,Excellent quality!,I got the cables yesterday and immediately put...,5,B0D49CWPH2,I got cables yesterday immediately put work. S...,0.000,0.722,0.278,0.9562,1
3,3,3,3,CHARGES SO FAST,So cheap and charges so fast. I got these to c...,5,B0D49CWPH2,"So cheap charges fast. I got charge laptop, ip...",0.041,0.763,0.196,0.8363,1
4,4,4,4,Great Quality Charging Cables!!,Extremely pleased with the high quality of the...,5,B0D49CWPH2,Extremely pleased high quality charging cables...,0.000,0.659,0.341,0.9482,1


In [5]:
# Separate the data into labels and features
# Separate the X variable, the features
X = merged_df['comments_no_stopwords']
# Separate the y variable, the labels
y = merged_df['sentiment']
Counter(y)

Counter({1: 2631, 0: 248})

In [6]:
vectorizer = CountVectorizer(analyzer='word', max_features=8445)
X_vector= vectorizer.fit_transform(X).toarray()
X_vector.shape

(2879, 8445)

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_vector)
X_tfidf.shape

(2879, 8445)

In [8]:
# Split the data using train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.25, random_state=0)

# custom split data for training and testing
N = 220
pos_train = y[y == 1].sample(N, random_state=1)
neg_train = y[y == 0].sample(N, random_state=1)

# sample positive and negative for validation/testing
y_test_pos = y[(y==1) & ~(y.index.isin(pos_train.index))].sample(248 - N, random_state=100)
y_test_neg = y[(y==0) & ~(y.index.isin(neg_train.index))].sample(248 - N, random_state=100)
y_test = pd.concat([y_test_pos, y_test_neg])

X_test = X_tfidf[y.index.isin(y_test.index)]

# use reverse index from test for training
X_train = X_tfidf[~y.index.isin(y_test.index)]
y_train = y[~y.index.isin(y_test.index)]


In [9]:
# Instantiate the Logistic Regression model
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier() 

# Fit the model using training data
logistecModel = model.fit(X_train, y_train)

# Make a prediction using the testing data
y_predicted = logistecModel.predict(X_test)

# Calculate the accuracy
print('Accuracy: %.2f%%' % (accuracy_score(y_predicted, y_test)*100))

Accuracy: 66.07%


In [10]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, y_predicted)

print(cm)

# Print the classification report for the model
print(classification_report(y_test, y_predicted))

[[11 17]
 [ 2 26]]
              precision    recall  f1-score   support

           0       0.85      0.39      0.54        28
           1       0.60      0.93      0.73        28

    accuracy                           0.66        56
   macro avg       0.73      0.66      0.63        56
weighted avg       0.73      0.66      0.63        56

